**Import Statements:** These lines import the necessary libraries: NumPy for numerical operations, Pandas for data manipulation, Scikit-learn for data preprocessing and evaluation metrics, and TensorFlow's Keras for building and training neural networks.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

Data Generation Functions:

In [2]:
# Generate synthetic data for mild steel plates
def generate_mild_steel_data(n_samples=1000):
    np.random.seed(42)

    carbon = np.random.uniform(0.1, 0.5, n_samples)
    manganese = np.random.uniform(0.3, 1.5, n_samples)
    silicon = np.random.uniform(0.1, 0.5, n_samples)
    phosphorus = np.random.uniform(0.01, 0.05, n_samples)
    sulfur = np.random.uniform(0.01, 0.05, n_samples)
    tensile_strength = np.random.uniform(350, 650, n_samples)
    yield_strength = np.random.uniform(200, 450, n_samples)
    elongation = np.random.uniform(15, 30, n_samples)

    quality = np.where((tensile_strength > 500) & (yield_strength > 350) & (elongation > 22), 1, 0)

    data = pd.DataFrame({
        'Carbon': carbon,
        'Manganese': manganese,
        'Silicon': silicon,
        'Phosphorus': phosphorus,
        'Sulfur': sulfur,
        'Tensile_Strength': tensile_strength,
        'Yield_Strength': yield_strength,
        'Elongation': elongation,
        'Quality': quality
    })

    return data

In [3]:
# Generate synthetic data for stainless steel
def generate_stainless_steel_data(n_samples=500):
    np.random.seed(43)

    carbon = np.random.uniform(0.01, 0.15, n_samples)
    chromium = np.random.uniform(16, 26, n_samples)
    nickel = np.random.uniform(6, 22, n_samples)
    manganese = np.random.uniform(0.5, 2, n_samples)
    silicon = np.random.uniform(0.1, 1, n_samples)
    phosphorus = np.random.uniform(0.01, 0.05, n_samples)
    sulfur = np.random.uniform(0.01, 0.03, n_samples)
    tensile_strength = np.random.uniform(500, 800, n_samples)
    yield_strength = np.random.uniform(200, 500, n_samples)
    elongation = np.random.uniform(20, 40, n_samples)

    quality = np.where((tensile_strength > 650) & (yield_strength > 350) & (elongation > 30), 1, 0)

    data = pd.DataFrame({
        'Carbon': carbon,
        'Chromium': chromium,
        'Nickel': nickel,
        'Manganese': manganese,
        'Silicon': silicon,
        'Phosphorus': phosphorus,
        'Sulfur': sulfur,
        'Tensile_Strength': tensile_strength,
        'Yield_Strength': yield_strength,
        'Elongation': elongation,
        'Quality': quality
    })

    return data

Functions above are placeholders for our data generation code. They should return datasets for mild steel and stainless steel respectively.

Function below should take the raw data, preprocess it, split it into training and testing sets, and scale the features. It's expected to return X_train, X_test, y_train, y_test, and the scaler object.

In [4]:
# Prepare data for model training
def prepare_data(data):
    X = data.drop('Quality', axis=1)
    y = data['Quality']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    return X_train_scaled, X_test_scaled, y_train, y_test, scaler

**Model Building and Training Function:** This function builds a neural network model with three hidden layers (64, 32, and 16 neurons) and an output layer. It uses ReLU activation for hidden layers and sigmoid for the output layer. The model is compiled with Adam optimizer and binary crossentropy loss. It's then trained for 50 epochs with a batch size of 32 and 20% validation split.

In [5]:
# Build and train the model for mild steel
def build_and_train_model(X_train, y_train, X_test, y_test):
    model = Sequential([
        Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
        Dropout(0.3),
        Dense(32, activation='relu'),
        Dropout(0.3),
        Dense(16, activation='relu'),
        Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

    history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=0)

    return model

**Transfer Learning Function**

In [13]:
def transfer_learning(base_model, X_train, y_train, X_test, y_test):
    # Create a new model
    new_model = Sequential()

    # Add a new input layer to handle the different input shape
    new_model.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],)))

    # Add the layers from the base model, excluding the input and output layers
    for layer in base_model.layers[1:-1]:
        new_model.add(layer)

    # Add a new output layer
    new_model.add(Dense(1, activation='sigmoid'))

    # Freeze the weights of the transferred layers
    for layer in new_model.layers[1:-1]:
        layer.trainable = False

    new_model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

    history = new_model.fit(X_train, y_train, epochs=30, batch_size=32, validation_split=0.2, verbose=0)

    return new_model

# Main execution
if __name__ == "__main__":
    # Generate and prepare mild steel data
    mild_steel_data = generate_mild_steel_data()
    X_train_ms, X_test_ms, y_train_ms, y_test_ms, scaler_ms = prepare_data(mild_steel_data)

    # Build and train model on mild steel data
    mild_steel_model = build_and_train_model(X_train_ms, y_train_ms, X_test_ms, y_test_ms)

    # Evaluate mild steel model
    ms_predictions = (mild_steel_model.predict(X_test_ms) > 0.5).astype(int)
    ms_accuracy = accuracy_score(y_test_ms, ms_predictions)
    print("Mild Steel Model Accuracy:", ms_accuracy)
    print("Mild Steel Classification Report:")
    print(classification_report(y_test_ms, ms_predictions))

    # Generate and prepare stainless steel data
    stainless_steel_data = generate_stainless_steel_data()
    X_train_ss, X_test_ss, y_train_ss, y_test_ss, scaler_ss = prepare_data(stainless_steel_data)

    # Perform transfer learning for stainless steel
    stainless_steel_model = transfer_learning(mild_steel_model, X_train_ss, y_train_ss, X_test_ss, y_test_ss)

    # Evaluate stainless steel model
    ss_predictions = (stainless_steel_model.predict(X_test_ss) > 0.5).astype(int)
    ss_accuracy = accuracy_score(y_test_ss, ss_predictions)
    print("\nStainless Steel Model Accuracy:", ss_accuracy)
    print("Stainless Steel Classification Report:")
    print(classification_report(y_test_ss, ss_predictions))

    # Print model summaries
    print("\nMild Steel Model Summary:")
    mild_steel_model.summary()

    print("\nStainless Steel Model Summary:")
    stainless_steel_model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Mild Steel Model Accuracy: 0.97
Mild Steel Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.99      0.98       181
           1       0.88      0.79      0.83        19

    accuracy                           0.97       200
   macro avg       0.93      0.89      0.91       200
weighted avg       0.97      0.97      0.97       200



/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step

Stainless Steel Model Accuracy: 0.13
Stainless Steel Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        87
           1       0.13      1.00      0.23        13

    accuracy                           0.13       100
   macro avg       0.07      0.50      0.12       100
weighted avg       0.02      0.13      0.03       100


Mild Steel Model Summary:


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_10 (Dense)                     │ (None, 64)                  │             704 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 16)                  │             528 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_11 (Dense)                     │ (None, 1)                   │              17 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,773 (18.65 KB)

 Trainable params: 721 (2.82 KB)

 Non-trainable params: 2,608 (10.19 KB)

 Optimizer params: 1,444 (5.64 KB)

Function below performs transfer learning. It creates a new model, copies the architecture of the base model (except for the first and last layers), freezes the middle layers, and then trains this new model on the new data. The learning rate is reduced for fine-tuning.

In [6]:
# Perform transfer learning for stainless steel
def transfer_learning(base_model, X_train, y_train, X_test, y_test):
    for layer in base_model.layers[:-1]:
        layer.trainable = False

    new_model = Sequential(base_model.layers[:-1])
    new_model.add(Dense(1, activation='sigmoid'))

    new_model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

    history = new_model.fit(X_train, y_train, epochs=30, batch_size=32, validation_split=0.2, verbose=0)

    return new_model

This block executes the main workflow:
- It generates and prepares data for mild steel.
- Builds and trains a model on mild steel data.
- Evaluates the mild steel model and prints its accuracy and classification report.
- Generates and prepares data for stainless steel.
- Performs transfer learning using the mild steel model as a base for the stainless steel model.
- Evaluates the stainless steel model and prints its accuracy and classification report.
- Finally, it prints summaries of both models.

In [7]:
# Main execution
if __name__ == "__main__":
    # Generate and prepare mild steel data
    mild_steel_data = generate_mild_steel_data()
    X_train_ms, X_test_ms, y_train_ms, y_test_ms, scaler_ms = prepare_data(mild_steel_data)

In [8]:
    # Build and train model on mild steel data
    mild_steel_model = build_and_train_model(X_train_ms, y_train_ms, X_test_ms, y_test_ms)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [9]:
    # Evaluate mild steel model
    ms_predictions = (mild_steel_model.predict(X_test_ms) > 0.5).astype(int)
    ms_accuracy = accuracy_score(y_test_ms, ms_predictions)
    print("Mild Steel Model Accuracy:", ms_accuracy)
    print("Mild Steel Classification Report:")
    print(classification_report(y_test_ms, ms_predictions))

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
Mild Steel Model Accuracy: 0.97
Mild Steel Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.98      0.98       181
           1       0.84      0.84      0.84        19

    accuracy                           0.97       200
   macro avg       0.91      0.91      0.91       200
weighted avg       0.97      0.97      0.97       200



In [10]:
    # Generate and prepare stainless steel data
    stainless_steel_data = generate_stainless_steel_data()
    X_train_ss, X_test_ss, y_train_ss, y_test_ss, scaler_ss = prepare_data(stainless_steel_data)

In [14]:
    # Perform transfer learning for stainless steel
    stainless_steel_model = transfer_learning(mild_steel_model, X_train_ss, y_train_ss, X_test_ss, y_test_ss)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [15]:
    # Evaluate stainless steel model
    ss_predictions = (stainless_steel_model.predict(X_test_ss) > 0.5).astype(int)
    ss_accuracy = accuracy_score(y_test_ss, ss_predictions)
    print("\nStainless Steel Model Accuracy:", ss_accuracy)
    print("Stainless Steel Classification Report:")
    print(classification_report(y_test_ss, ss_predictions))

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step

Stainless Steel Model Accuracy: 0.87
Stainless Steel Classification Report:
              precision    recall  f1-score   support

           0       0.87      1.00      0.93        87
           1       0.00      0.00      0.00        13

    accuracy                           0.87       100
   macro avg       0.43      0.50      0.47       100
weighted avg       0.76      0.87      0.81       100



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
